In [30]:
import pandas as pd
import spacy

In [31]:
processed_text = pd.read_json("processed_text.json")
processed_text_tf_idf = pd.read_json("processed_text_tf_idf.json")

In [32]:
processed_text_tf_idf.head()

,title,text,url,processed_text,tfidf_vector
0,Pandemic,"A pandemic (from Greek πᾶν, pan, ""all"" and δῆμ...",https://en.wikipedia.org/wiki/Pandemic,"[pandemic, greek, pan, demo, people, epidemic,...","{'pandemic': 0.1814556331, 'greek': 26.0, 'pan..."
1,Epidemiology of HIV/AIDS,"HIV/AIDS, or Human Immunodeficiency Virus, is ...",https://en.wikipedia.org/wiki/Epidemiology_of_...,"[hiv, aid, human, immunodeficiency, virus, con...","{'pandemic': 0.0518444666, 'greek': 0.0, 'pan'..."
2,Antonine Plague,"The Antonine Plague of 165 to 180 AD, also kno...",https://en.wikipedia.org/wiki/Antonine_Plague,"[plague, ad, know, plague, galen, galen, physi...","{'pandemic': 0.0518444666, 'greek': 0.0, 'pan'..."
3,Basic reproduction number,"In epidemiology, the basic reproduction number...",https://en.wikipedia.org/wiki/Basic_reproducti...,"[epidemiology, basic, reproduction, number, ba...","{'pandemic': 0.0, 'greek': 0.0, 'pan': 0.0, 'd..."
4,Bills of mortality,Bills of mortality were the weekly mortality s...,https://en.wikipedia.org/wiki/Bills_of_mortality,"[bill, mortality, weekly, mortality, statistic...","{'pandemic': 0.0, 'greek': 0.0, 'pan': 0.0, 'd..."


In [35]:
# Create dictionary 
dict.fromkeys([word for document in processed_text_tf_idf.processed_text for word in document], [])

{'pandemic': [],
 'greek': [],
 'pan': [],
 'demo': [],
 'people': [],
 'epidemic': [],
 'infectious': [],
 'disease': [],
 'spread': [],
 'large': [],
 'region': [],
 'instance': [],
 'multiple': [],
 'continent': [],
 'worldwide': [],
 'affect': [],
 'substantial': [],
 'number': [],
 'widespread': [],
 'endemic': [],
 'stable': [],
 'infected': [],
 'recurrence': [],
 'seasonal': [],
 'influenza': [],
 'generally': [],
 'exclude': [],
 'occur': [],
 'simultaneously': [],
 'globe': [],
 'human': [],
 'history': [],
 'smallpox': [],
 'tuberculosis': [],
 'fatal': [],
 'record': [],
 'black': [],
 'death': [],
 'know': [],
 'plague': [],
 'kill': [],
 'estimate': [],
 'million': [],
 'century': [],
 'term': [],
 'later': [],
 'include': [],
 'spanish': [],
 'flu': [],
 'current': [],
 'sar': [],
 'hiv': [],
 'aid': [],
 'immunodeficiency': [],
 'virus': [],
 'consider': [],
 'author': [],
 'global': [],
 'currently': [],
 'use': [],
 'describe': [],
 'approximately': [],
 'infect': [],

In [29]:
for i in processed_text_tf_idf.tfidf_vector[0]:
    print(len(i))

1285


In [25]:
len(processed_text_tf_idf.processed_text[0])

80